## imports

In [190]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
from astropy.io.fits import getheader
from datetime import datetime
from astropy.io.fits.header import Header
#see guide to create fits files: https://pyfits.readthedocs.io/en/latest/#creating-a-new-fits-file
#Should I write this as a function or as a program?
#If i write it as a function it can be imported into any python script
#if i write it as a program it could run more smoothly, but is very rigid compared to the function implementation

## NOTES
    -Going to make a function that takes input strings and arrays, then outputs a fits file

## Reading Example fits file
    -opening ASIM pha file and check out headers and how the data is structured
    - load the data into arrays or tables (dunno which type of object the data will be in)

In [6]:
HDUlist = fits.open("exPHA.fits")
print("There are ", len(HDUlist), " objects in the HDUlist \n ",HDUlist)
#contains the fits.hdu.image.PrimaryHDU object which stores the information about the burst & spacecraft
# also contains fits.hdu.table.BinTableHDU which 

There are  4  objects in the HDUlist 
  [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f9e1cfc9f98>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e1cfcd5c0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e1cfcdfd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e1cfd4940>]


How the exPHA file looks in fv in xspec:
![alt text](hdr.png)

Should format the ASIM primary header to be similar to the GBM header:
![alt text](gbmPrimaryHDR.png)

Obviously paramerters that needs to be corrected for each burst...

Parameters that needs to be corrected each time of creation:
    DATE
    DATE-OBS
    DATE-END
    TSTART
    TSTOP
    TRIGTIME
    OBJECT
    RA_OBJ
    DEC_OBJ
    FILENAME
    
Redundant information:
    FILE-VER
    DETNAM
    OBSERVER
    MJDREFI
    MJDREFF
    ERR_RAD
    INFILE01
    CHECKSUM
    DATASUM
    

In [22]:
hdr = getheader("exPHA.fits",1)
hdr["EXTNAME"] = "SPECTRUM" #hdr works as a dictionary which is a mutable object
hdr["EXTNAME"]

'SPECTRUM'

## Primary HDR template

In [479]:
gbmHDU = fits.open("exGBM.pha")
gbmHDR = getheader("exGBM.pha",0)
type(gbmHDU)
gbmHDU

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7f9e0a7f2748>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e09aa3978>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e09495080>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7f9e096ac278>]

exGBM b0 HDR:

![alt text](gbmHDR.png)

In [165]:
redundant_keys = [
    "INFILE01",
    "CHECKSUM",
    "DATASUM",
    "DETNAM",
    "DATE-END",
    "DATE-OBS",
    "TIMESYS",
    "TIMEUNIT",
    "TRIGTIME",
    "FILE-VER",
    "MJDREFI",
    "MJDREFF",
    "OBSERVER"]
gbmHDR_copy = gbmHDR
for redK in redundant_keys:
    del gbmHDR_copy[redK]

In [241]:
utcTime= datetime.utcnow().isoformat(timespec="seconds")
gbmHDR_copy["CREATOR"] = "ASIM Fits v1.0.0"
gbmHDR_copy.comments["CREATOR"] = "Program name. Written by Andreas Ramsli"
gbmHDR_copy["TELESCOP"] = "ASIM"
gbmHDR_copy["INSTRUME"] = "MXGS: HED"
gbmHDR_copy["ORIGIN"] = "University of Bergen"
gbmHDR_copy["DATE"] = utcTime
gbmHDR_copy.comments["DATE"] = "file creation date (YYYY-MM-DDThh:mm:ss UT)"
gbmHDR_copy["T0"] = ""
gbmHDR_copy["TSTART"] = ""
gbmHDR_copy.comments["TSTART"] = "[s] TSTART given relative to T0"
gbmHDR_copy["TSTOP"] = ""
gbmHDR_copy.comments["TSTOP"] = "[s] TSTOP given relative to T0"
gbmHDR_copy.comments["OBJECT"] = "Burst name in standard format, GRB yymmdd"
gbmHDR_copy["FILENAME"] = ""
gbmHDR_copy["ERR_RAD"] = ""
gbmHDR_copy["OBJECT"] = ""
gbmHDR_copy["RA_OBJ"] = ""
gbmHDR_copy["DEC_OBJ"] = ""

In [243]:
primaryKeys = []
primaryValues = []
primaryComments = []

for k in gbmHDR_copy.keys():
    primaryKeys.append(k)
    primaryComments.append(gbmHDR_copy.comments[k])
for v in gbmHDR_copy.values():
    primaryValues.append(v)

In [252]:
for i,k in enumerate(gbmHDR_copy.keys()):
    if k != primaryKeys[i]:
        print(i,primaryKeys[i])

16 RA-OBJ
17 DEC-OBJ
18 ERR-RAD


In [257]:
primaryKeys = ['SIMPLE','BITPIX','NAXIS','EXTEND','CREATOR','FILETYPE','TELESCOP','INSTRUME','ORIGIN','DATE','TSTART','TSTOP','FILENAME','OBJECT','RADECSYS','EQUINOX','RA-OBJ','DEC-OBJ','ERR-RAD','T0']
primaryValues= [True,8,0,True,'ASIM Fits v1.0.0','SPECTRUM','ASIM','HED','University of Bergen','2022-09-28T11:46:10','','','','','FK5',2000.0,'','','','']
primaryComments = ['conforms to FITS standard','array data type','number of array dimensions','','Program name. Written by Andreas Ramsli','Name for this type of FITS file','Name of mission/satellite',
 'Specific instrument used for observation','Name of institution making file','file creation date (YYYY-MM-DDThh:mm:ss UT)','[s] TSTART given relative to T0','[s] TSTOP given relative to T0','Name of this file',
                   'Burst name in standard format, GRB yymmdd','Stellar reference frame','Equinox for RA and Dec','Calculated RA of burst','Calculated Dec of burst','Calculated Location Error Radius','Trigger time given as YYYY-MM-DD hh:mm:ss.f']

primaryDict = {}
for i, key in enumerate(primaryKeys):
    primaryDict[key]=primaryValues[i],primaryComments[i]

## Writing example data into a new fits file
    -load the data retrieved above into a new fits file
    -remove redundant information in the primary header

### Primary HDR

In [553]:
#Generate a new primary header from the primaryDict
primaryHDU = fits.PrimaryHDU([0])
primaryHDR = primaryHDU.header
for key,val in primaryDict.items():
    primaryHDR[key] = val[0]
    primaryHDR.comments[key] = val[1]
primaryHDR.comments["T0"]
primaryHDR
#TODO: Add in DATE-OBS TIME-OBS DATE-END TIME-END

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
NAXIS1  =                    1                                                  
EXTEND  =                    T                                                  
CREATOR = 'ASIM Fits v1.0.0'   / Program name. Written by Andreas Ramsli        
FILETYPE= 'SPECTRUM'           / Name for this type of FITS file                
TELESCOP= 'ASIM    '           / Name of mission/satellite                      
INSTRUME= 'HED     '           / Specific instrument used for observation       
ORIGIN  = 'University of Bergen' / Name of institution making file              
DATE    = '2022-09-28T11:46:10' / file creation date (YYYY-MM-DDThh:mm:ss UT)   
TSTART  = '' / [s] TSTART given relative to T0                                  
TSTOP   = '' / [s] TSTOP giv

### Sectrum HDR

In [ ]:
redSpecKeys = [""]
#change NAXIS1=18 (?), ANCRFILE=NONE, CHANTYP=PHA, Comment on TSTART/STOP, comment on EXPOSURE, HUDVERS='1.3.0'

#check: TSCAL2, TZERO2

#add from KW: DATE-OBS,TIME-OBS,DATE-END,TIME-END
#TLMIN2= 1 / The first channel in the response              
#TLMAX2=40 / The last channel in the response 

In [260]:
#Generating a template header for SPECTRUM

#getheader("exPHA.fits",1)

In [259]:
#getheader("exKW.pha",1)

In [284]:
specKeys = []
specValues = []
specComments = []

asimHDR = getheader("exPHA.fits",1)

for k in asimHDR.keys():
    specKeys.append(k)
    specComments.append(asimHDR.comments[k])
for v in asimHDR.values():
    specValues.append(v)

specDict = {}
for i, key in enumerate(specKeys):
    specDict[key]=specValues[i],specComments[i]

In [466]:
df = pd.DataFrame(data=[specKeys,specValues,specComments])

df = df.T
df.iloc[0,2] = "binary table extension"
df.iloc[1,2] = "8-bit bytes"
df.iloc[2,2] = "2-dimensional binary table"
df.iloc[3,1] = 16 #bytes pr row
df.iloc[5,2] = "size of special data area"
df.iloc[7,1] = 3
df.iloc[11,2] = 'File creation date (UT)'
df.iloc[11,1] = 'none' #del date
df.iloc[13,1] = 'none'
df.iloc[13,2] = '[s] Start time of spectrum accumulation relative to T0'
df.iloc[14,1] = 'none'
df.iloc[15,1] = 'none'
df.iloc[14,2] = '[s] End time of spectrum accumulation relative to T0'
df.iloc[19,1] = 'none' #bakfile
df.iloc[21,1] = 'none' #respfile
df.iloc[21,1] = 'none'
df.iloc[22,1] = 'none'
df.iloc[23,1] = 'none'
df.iloc[23,2] = 'Instrument filter in use'
df.iloc[24,1] = 'PI'
df.iloc[25,1] = 'T'
df.iloc[28,1] = '1.3.0'
df.iloc[28,2] = 'Version of format (OGIP memo OGIP-92-007)'
df.iloc[31,1] = '1I'
df.iloc[31,2] = 'data format of field: 2-byte INTEGER'
df.iloc[32,1] = '1E'
df.iloc[32,2] = 'data format of field: 4-byte INTEGER'
df.iloc[33,1] = '1E'
df.iloc[33,2] = 'data format of field: 4-byte INTEGER'
df.iloc[35,2] = 'label for field   1'
df.iloc[36,2] = 'label for field   2'
df.iloc[38,2] = 'label for field   3'
df.drop(index=[29,30,34,37,39,40,41,42,43,44,45,46,47,48,49],inplace=True)

df

,0,1,2
0,XTENSION,BINTABLE,binary table extension
1,BITPIX,8,8-bit bytes
2,NAXIS,2,2-dimensional binary table
3,NAXIS1,16,Number of bytes per row
4,NAXIS2,40,Number of rows
5,PCOUNT,0,size of special data area
6,GCOUNT,1,Required value
7,TFIELDS,3,Number of columns in table
8,EXTNAME,SPECTRUM,Extension name
9,TELESCOP,ASIM,Telescope or mission name


In [472]:
#need to figure out which keywords are missing from the manditory list
asimKeys = df[0].to_numpy()
not_in = []
for k in hearsecKeywords:
    if k not in asimKeys:
        not_in.append(k)
not_in.pop(0) #pop: TUNIT2
print(len(not_in),len(hearsecKeywords),len(asimKeys))

17 51 35


In [468]:
not_in

['TUNIT2',
 'TLMIN1',
 'TLMAX1',
 'STAT_ERR',
 'SYS_ERR',
 'QUALITY',
 'DATAMODE',
 'XFLT0001',
 'OBJECT',
 'USER',
 'ORIGIN',
 'EQUINOX',
 'RADECSYS',
 'DATE-OBS',
 'TIME-OBS',
 'DATE-END',
 'TIME-END',
 'TOTCTS']

In [415]:
#getheader("exPHA.fits",1)

In [408]:
#getheader("exGBM.pha",2)

In [417]:
#getheader("exKW.pha",1)

In [ ]:
# important keywords to use in spectrum header: DATE-OBS, TIME-OBS, DATE-END, TIME-END, CREATOR

In [254]:
#pass a dictionary into the function to assign the values for the availible keys. If the key is not found, it will be added
#must pass one dict for primary header and one for the spectrum header

In [219]:
hdul = fits.HDUList([primaryImg])
hdul.writeto('new.fits')

## HEARSEC example PHA header

In [359]:
hearsecKeywords=["XTENSION","BITPIX","NAXIS","NAXIS1","NAXIS2","PCOUNT","GCOUNT","TFIELDS","TTYPE1","TFORM1","TTYPE2","TFORM2","TUNIT2",
           "TTYPE3","TFORM3","EXTNAME","TLMIN1","TLMAX1","POISSERR","STAT_ERR","SYS_ERR","QUALITY","DETCHANS",
                  "EXPOSURE","AREASCAL","CORRSCAL","BACKSCAL","BACKFILE","CORRFILE","RESPFILE","ANCRFILE","CHANTYPE","HDUCLASS","HDUCLAS1",
                  "HDUVERS","TELESCOP","INSTRUME","FILTER","DATAMODE","XFLT0001","OBJECT","USER","ORIGIN","DATE","EQUINOX","RADECSYS",
                "DATE-OBS","TIME-OBS","DATE-END","TIME-END","TOTCTS"]

### PHA file header creation

In [470]:
not_inDict={}
for key in not_in:
    not_inDict[key] = "none","none"
not_inDict['TLMIN1'] = 1,'Lowest legal channel number'
not_inDict['TLMAX1'] = 40,'Highest legal channel number'
not_inDict['STAT_ERR'] = 0,'no statistical error specified'
not_inDict['SYS_ERR'] = 0, 'no systematic error specified'
not_inDict['QUALITY'] = 0,'no data quality information specified'
not_inDict['DATAMODE'] ='PH','Datamode'
not_inDict['XFLT0001'] ='none    ','XSPEC selection filter description' #replace all NONE with 'none    '
not_inDict['OBJECT'] ='none    ','Name of observed object'
not_inDict['USER'] ='none    ','User name of creator'
not_inDict['ORIGIN'] ='none    ','Name of institution making the fits file'
not_inDict['EQUINOX'] =2000.0,'Equinox of celestial coord system'
not_inDict['RADECSYS'] ='FK5','coord frame used for EQUINOX'
not_inDict['DATE-OBS'] ='none    ','Date observations were made (yyyy-mm-dd)'
not_inDict['TIME-OBS'] ='none    ','Time observations were made (hh:mm:ss)'
not_inDict['DATE-END'] ='none    ','Date observations ended (yyyy-mm-dd)'
not_inDict['TIME-END'] ='none    ','Time observations were made (hh:mm:ss)'
not_inDict['TOTCTS'] ='none    ','Total counts in spectrum'
not_inDict['HDUCLAS2'] = 'TOTAL','indicating the data stored (TOTAL: gross PHA spectrum (source+bkg)'
not_inDict['HUDCLAS3'] = 'COUNT', 'indicating further the data stored (COUNT rather than count/s)'
not_inDict['CREATOR'] = 'ASIM fits v1.0.0','Program name and version. Written by Andreas Ramsli'
not_inDict['HDUCLAS4'] = 'TYPE:I','indicating whether this is a type I or II extension. TYPE:I for signle spectrum'

In [471]:
#concat df_not_inDict to df
#first make a df out of it
not_keys = []
not_values = []
not_comments = []
for key,value in not_inDict.items():
    not_keys.append(key)
    not_values.append(value[0])
    not_comments.append(value[1])
    
df_notIN = pd.DataFrame(data=[not_keys,not_values,not_comments])
df_notIN = df_notIN.T
fullDF = pd.concat([df,df_notIN])
fullDF.sort_values(by=[0],inplace=True,ignore_index=True)
fullDF.reset_index(inplace=True)
fullDF.drop(labels="index",axis=1,inplace=True)
fullDF
#TODO: extract the keywords and create a new clean df, and the check it against the ogip_92_007 paper


,0,1,2
0,ANCRFILE,none,ancillary response
1,AREASCAL,1,"Area scaling factor, no field AREASCAL"
2,BACKFILE,none,Background FITS file
3,BACKSCAL,1,"Background scale factor, no field BACKSCAL"
4,BITPIX,8,8-bit bytes
5,CHANTYPE,PI,"channel type (PHA, PI etc)"
6,CORRFILE,NONE,Correlation FITS file
7,CORRSCAL,0,Correlation scale factor
8,CREATOR,ASIM fits v1.0.0,Program name and version. Written by Andreas R...
9,DATAMODE,PH,Datamode


# SPECTRUM PHA

### HDR for SPECTRUM

In [487]:
#now all the keywords,values & comments are in place for the PHA HDR
#next: 
#load it into a fits file
#create the templace for inputting the information required
specKeys,specValues,specComments = fullDF[0].to_numpy(),fullDF[1].to_numpy(),fullDF[2].to_numpy()
specDict = {}
for i,key in enumerate(specKeys):
    specDict[key] = specValues[i],specComments[i]
    

#SPECTRUM should be of the type: astropy.io.fits.hdu.table.BinTableHDU (containing both hdr and table)
#type of data table: astropy.io.fits.fitsrec.FITS_rec

In [482]:
data = gbmHDU[2].data
type(data)

astropy.io.fits.fitsrec.FITS_rec

In [549]:
#creating the header for the SPECTRUM HDU

specHDR = fits.Header()
for key,val in specDict.items():
    specHDR[key] = val[0]
    specHDR.comments[key] = val[1]

### Table creation for SPECTRUM

In [570]:
#specifying example arrays for columns
countsArray = np.array([0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   1,  23,  35, 110,  91,  76,  56,  53,  35,  17,  14,   9,
         9,   3,   5,   3,   0,   4,   4,   2,   1,   9,   3,   0,   0,
         0,   0], dtype=np.int16)
statErrArray = np.array([0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
        0.       ,  0.       ,  0.       ,  0.       ,  1.       ,
        4.7958317,  5.91608  , 10.488089 ,  9.539392 ,  8.717798 ,
        7.483315 ,  7.28011  ,  5.91608  ,  4.1231055,  3.7416575,
        3.       ,  3.       ,  1.7320508,  2.236068 ,  1.7320508,
        0.       ,  2.       ,  2.       ,  1.4142135,  1.       ,
        3.       ,  1.7320508,  0.       ,  0.       ,  0.       ,
        0.       ], dtype=np.float32)

In [573]:
channelColumn = fits.Column(name=specHDR['TTYPE1'],format=specHDR['TFORM1'], array=np.arange(1,len(countsArray)+1))
countsColumn = fits.Column(name=specHDR['TTYPE2'],format=specHDR['TFORM2'], array=countsArray)
statErrColumn = fits.Column(name=specHDR['TTYPE3'],format=specHDR['TFORM3'], array=statErrArray)

#check if this works, HDR
specTableHDU = fits.BinTableHDU.from_columns(columns=[channelColumn,countsColumn,statErrColumn],header=specHDR) 

In [578]:
#check how it behaves now

hdul = fits.HDUList([primaryHDU,specTableHDU])
hdul.writeto('onlySpec.fits')

#behaves well

In [ ]:
len(countsArray),len(statErrArray),len(np.arange(1,len(countsArray)+1))

(40, 40, 40)

### EBOUNDS HDR
    -Only a selected keywords are need and must be modified accordingly
    -Seems like most of the keywords are still in, but some have been modified to EBOUNDS parameters. See diff in GBM and HED_PHA

In [624]:
#Use exGBM header as template
exGBMHDR = fits.getheader("exGBM.pha",1) #much information which is also contained in the SPECTRUM HDR. Don't need to declare it again.
exPHAHDR = fits.getheader("exPHA.fits",2) #Using this as a template


eboundsHDR = fits.Header(cards=exPHAHDR.cards)
eboundsHDR["DATE"] = 'none'

In [629]:
eboundsHDRkeys = []
eboundsHDRvalues = []
eboundsHDRcomments = []

for row in eboundsHDR.cards:
    eboundsHDRkeys.append(row[0])
    eboundsHDRvalues.append(row[1])
    eboundsHDRcomments.append(row[2]) 
eboundsHDRcomments[17] = 'Highest legal channel number'
eboundsHDRcomments[16] = 'Lowest legal channel number'
eboundsHDRvalues[17] = 40
eboundsHDRvalues[16] = 1

In [634]:
eboundsHDR = fits.Header()

for i,key in enumerate(eboundsHDRkeys):
    eboundsHDR[key] = eboundsHDRvalues[i],eboundsHDRcomments[i]
eboundsHDR

XTENSION= 'BINTABLE'           / Binary table written by MWRFITS v1.4           
BITPIX  =                    8 / Required value                                 
NAXIS   =                    2 / Required value                                 
NAXIS1  =                   10 / Number of bytes per row                        
NAXIS2  =                   40 / Number of rows                                 
PCOUNT  =                    0 / Normally 0 (no varying arrays)                 
GCOUNT  =                    1 / Required value                                 
TFIELDS =                    3 / Number of columns in table                     
EXTNAME = 'EBOUNDS '           / Extension name                                 
TELESCOP= 'ASIM    '           / Telescope or mission name                      
INSTRUME= 'MXGS: HED'          / Instrument name                                
DATE    = 'none    '           / Creation date                                  
DETCHANS=                   

### EBOUNDS table

In [642]:
eminArray = np.array([1.000000e+01, 1.258925e+01, 1.584893e+01, 1.995262e+01,
       2.511886e+01, 3.162278e+01, 3.981072e+01, 5.011872e+01,
       6.309573e+01, 7.943282e+01, 1.000000e+02, 1.258925e+02,
       1.584893e+02, 1.995262e+02, 2.511886e+02, 3.162278e+02,
       3.981072e+02, 5.011872e+02, 6.309573e+02, 7.943282e+02,
       1.000000e+03, 1.258925e+03, 1.584893e+03, 1.995262e+03,
       2.511886e+03, 3.162278e+03, 3.981072e+03, 5.011872e+03,
       6.309573e+03, 7.943282e+03, 1.000000e+04, 1.258925e+04,
       1.584893e+04, 1.995262e+04, 2.511886e+04, 3.162278e+04,
       3.981072e+04, 5.011872e+04, 6.309573e+04, 7.943282e+04],
      dtype=np.float32)
emaxArray = np.array([1.258925e+01, 1.584893e+01, 1.995262e+01, 2.511886e+01,
       3.162278e+01, 3.981072e+01, 5.011872e+01, 6.309573e+01,
       7.943282e+01, 1.000000e+02, 1.258925e+02, 1.584893e+02,
       1.995262e+02, 2.511886e+02, 3.162278e+02, 3.981072e+02,
       5.011872e+02, 6.309573e+02, 7.943282e+02, 1.000000e+03,
       1.258925e+03, 1.584893e+03, 1.995262e+03, 2.511886e+03,
       3.162278e+03, 3.981072e+03, 5.011872e+03, 6.309573e+03,
       7.943282e+03, 1.000000e+04, 1.258925e+04, 1.584893e+04,
       1.995262e+04, 2.511886e+04, 3.162278e+04, 3.981072e+04,
       5.011872e+04, 6.309573e+04, 7.943282e+04, 1.000000e+05],
      dtype=np.float32)

In [648]:
eboundschannelColumn = fits.Column(name=eboundsHDR['TTYPE1'],format=eboundsHDR['TFORM1'], array=np.arange(1,len(eminArray)+1))
eboundMinColumn = fits.Column(name=eboundsHDR['TTYPE2'],format=eboundsHDR['TFORM2'], array=eminArray)
eboundMaxColumn = fits.Column(name=eboundsHDR['TTYPE3'],format=eboundsHDR['TFORM3'], array=emaxArray)

#check if this works, HDR
eboundsTableHDU = fits.BinTableHDU.from_columns(columns=[channelColumn,eboundMinColumn,eboundMaxColumn],header=eboundsHDR) 

### Joining the three HDU's

In [655]:
eboundsHDUL = fits.HDUList([primaryHDU,eboundsTableHDU,specTableHDU])
eboundsHDUL.writeto("almost_there.pha",checksum=True)

# Template HDR inputs

# Function for creating full fits file

### Template HDR's
    -Declare the neccecary information

### function

In [523]:
#function for creating the fits file
#inputs: primaryDict, spectrumDict, T0 (datetime), bins (np.array), counts (np.array), poisson error (np.array), duration (float [s])
#templateDict for ALL THREE HDRs implemented

#Check that the following keys have are provided: OBJECT, Ra, Dec etc.


#print_template=True should only do this to check what is given

#casting field 1 & 2 to integer

#In the example script that will produce the first fits files, should the TIME-START & END be given as a hh:mm:ss.f string?

#Key: Exposure is the duration of the spectrum accumulation or background accumulation time interval

#Keys that are used in more than one header is declared only once

#QUALITY table specified or specified after grppha? Should probably be specified first

#For information about the keywords and what they mean, check out the paper: ogip_92_007, provide link

#CHANTYPE: PHA or PI? The latter is pulse-invariant. See page 6 in ogip. Corrections have been applied, so i assume its PI

#function must check that the len of the arrays are equal

#function should check if ebounds bins are sorted.

#function should check if the channel number in specHDR is equal to the length of ebounds array.

# Example HDR

### Data Extension
    XTENSION= 'BINTABLE'           / binary table extension
    BITPIX  =                    8 / 8-bit bytes
    NAXIS   =                    2 / 2-dimensional binary table
    NAXIS1  =                    8 / width of table in bytes
    NAXIS2  =                 1024 / number of rows in table
    PCOUNT  =                    0 / size of special data area
    GCOUNT  =                    1 / one data group (required keyword)
    TFIELDS =                    3 / number of fields in each row
    TTYPE1  = 'CHANNEL '           / label for field   1
    TFORM1  = 'I       '           / data format of the field: 2-byte INTEGER
    TTYPE2  = 'COUNTS  '           / label for field   2
    TFORM2  = 'J       '           / data format of the field: 4-byte INTEGER
    TUNIT2  = 'count   '           / physical unit of field
    TTYPE3  = 'GROUPING'           / label for field   3
    TFORM3  = 'I       '           / data format of the field: 2-byte INTEGER
    EXTNAME = 'SPECTRUM'           / name of this binary table extension
    TLMIN1  =                    0 / Lowest legal channel number
    TLMAX1  =                 1023 / Highest legal channel number
    POISSERR=                    T / Poissonian errors to be assumed
    STAT_ERR=                    0 / no statistical error specified
    SYS_ERR =                    0 / no systematic error specified
    QUALITY =                    0 / no data quality information specified
    DETCHANS=                 1024 / Total No. of Detector Channels available

### Additional Mandatory keywords for XSPEC (in addition to those above)
    EXPOSURE=  2.91189522666037082672E+04 / Exposure time
    AREASCAL=  0.1000000000000E+01  /nominal effective area
    CORRSCAL=  0.1000000000000E+01  /correlation scale factor
    BACKSCAL=  0.2815246582031E-01  /background scale factor
    BACKFILE= 'back.sp '           / background FITS file for
    CORRFILE= 'none    '           / correlation FITS file for
    RESPFILE= 'none    '           / redistribution
    ANCRFILE= 'none    '           / ancillary response
    CHANTYPE= 'PI      '           / Channels assigned by detector electronics
    HDUCLASS= 'OGIP    '           / format conforms to OGIP standard
    HDUCLAS1= 'SPECTRUM'           / PHA dataset (OGIP memo OGIP-92-007)
    HDUVERS = '1.2.1   '           / Version of format (OGIP memo OGIP-92-007)
    TELESCOP= 'ASCA    '           / Telescope (mission) name
    INSTRUME= 'GIS2    '           / Instrument name
    FILTER  = 'none    '           / Instrument filter in use


### Additional Optional (but recommended) keywords
    DATAMODE= 'PH      '           / Datamode
    XFLT0001= 'none    '           / XSPEC selection filter description
    OBJECT  = 'BD +30 3639'        / Name of observed object
    USER    = 'kaa     '           / User name of creator
    FILIN001= 'screen.evt'         / Input file name
    ORIGIN  = 'NASA/GSFC'          / origin of fits file
    DATE    = '1995-02-07'         / FITS file creation date (yyyy-mm-dd)
    EQUINOX =               2000.0 / Equinox of celestial coord system
    RADECSYS= 'FK4     '           / coord frame used for EQUINOX
    DATE-OBS= '1994-10-29'         / Date observations were made (yyyy-mm-dd)
    TIME-OBS= '23:34:56'           / Time observations were made (hh:mm:ss)
    DATE-END= '1994-10-30'         / Date observations ended (yyyy-mm-dd)
    TIME-END= '18:30:56'           / Time observations ended (hh:mm:ss)
    HDUCLAS2= '        '           / WARNING This is NOT an OGIP-approved value
    HDUCLAS3= 'COUNT   '           / PHA data stored as Counts (not count/s)
    HDUVERS1= '1.2.1   '           / Obsolete - included for backwards compatibility
    TOTCTS  =                 1080 / Total counts in spectrum
    --------------------------------------------------------------------------

### Another example of Manditory keywords in PHA header
    EXTNAME (= SPECTRUM) - the name (i.e. type) of the extension
    TELESCOP - the "telescope" (i.e. mission/satellite name).
    INSTRUME - the instrument/detector.
    FILTER - the instrument filter in use (if any)
    EXPOSURE - the integration time (in seconds) for the PHA data (assumed to be corrected for deadtime, data drop-outs etc. )
    BACKFILE - the name of the corresponding background file (if any)
    BACKSCAL - the background scaling factor (unless included as a column).
    CORRFILE - the name of the corresponding correction file (if any)
    CORRSCAL - the correction scaling factor.
    RESPFILE - the name of the corresponding (default) redistribution matrix file (RMF; see George et al. 1992a).
    ANCRFILE - the name of the corresponding (default) ancillary response file (ARF; see George et al. 1992a).
    AREASCAL - the area scaling factor (unless included as a column).
    HDUCLASS - should contain the string "OGIP" to indicate that this is an OGIP style file.
    HDUCLAS1 - should contain the string "SPECTRUM" to indicate this is a spectrum.
    HDUVERS - the version number of the format (this document describes version 1.2.1)
    POISSERR - whether Poissonian errors are appropriate to the data (see below).
    CHANTYPE - whether the channels used in the file have been corrected in anyway (see below).
    DETCHANS - the total number of detector channels available.